## 配置文件

# 定义 Transformer 模型

In [1]:
'''注意力计算函数'''
import math
from torch.nn import functional as F
import torch

def attention(q, k, v, dropout_module = None, is_causal=False, dropout=0.0, mask=None):
    # 计算 QK^T / sqrt(d_k)，维度为 (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
    att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
    # 如果是解码器的 Casual LM，需要 mask 掉右上角的元素
    mask.unsqueeze_(1)
    if is_causal:
        # 生成一个下三角矩阵
        casual_mask = torch.tril(torch.ones(k.size(-2), k.size(-2)).view(1, 1, k.size(-2), k.size(-2)))
        # casual_mask 和原先的 attention_mask 做位运算
        mask = mask & casual_mask
    # 进行 Mask
    att = att.masked_fill(mask == False, float('-inf'))
    # 计算 softmax，维度为 (B, nh, T, T)
    att = F.softmax(att, dim=-1)
    # Attention Dropout
    if dropout_module is not None:
        att = dropout_module(dropout)
    # V * Score，维度为(B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
    y = att @ v 
    return y

In [2]:
# 测试一下 attention 计算

q = torch.rand((4, 2, 16, 8))
k = torch.rand((4, 2, 16, 8))
v = torch.rand((4, 2, 16, 8))
mask = torch.ones((4, 1, 16)) == 1
attention(q, k, v, mask=mask)

tensor([[[[0.4726, 0.5745, 0.5333,  ..., 0.5756, 0.5732, 0.4676],
          [0.4713, 0.5833, 0.5393,  ..., 0.5642, 0.5830, 0.4715],
          [0.4748, 0.5874, 0.5366,  ..., 0.5659, 0.5871, 0.4702],
          ...,
          [0.4789, 0.6027, 0.5426,  ..., 0.5696, 0.5915, 0.4665],
          [0.4833, 0.5881, 0.5361,  ..., 0.5788, 0.5763, 0.4724],
          [0.4814, 0.5825, 0.5447,  ..., 0.5661, 0.5783, 0.4679]],

         [[0.5026, 0.4728, 0.5219,  ..., 0.5615, 0.5012, 0.4368],
          [0.4890, 0.4949, 0.5211,  ..., 0.5829, 0.5007, 0.4404],
          [0.4982, 0.4879, 0.5207,  ..., 0.5733, 0.4955, 0.4312],
          ...,
          [0.5032, 0.4734, 0.5176,  ..., 0.5632, 0.5007, 0.4418],
          [0.5071, 0.4730, 0.5211,  ..., 0.5660, 0.4980, 0.4368],
          [0.4884, 0.5018, 0.5269,  ..., 0.5783, 0.4842, 0.4312]]],


        [[[0.5016, 0.4894, 0.4672,  ..., 0.5495, 0.3946, 0.6046],
          [0.5018, 0.4994, 0.4589,  ..., 0.5518, 0.3948, 0.5928],
          [0.5067, 0.4929, 0.4647,  ...,

In [3]:
import torch.nn as nn
import torch


'''多头注意力计算模块'''
class MultiHeadAttention(nn.Module):

    def __init__(self, config, is_causal=False):
        # 构造函数
        # config: 配置对象
        super().__init__()
        # 隐藏层维度必须是头数的整数倍
        assert config.n_embd % config.n_head == 0
        # Wq, Wk, Wv 参数矩阵，每个参数矩阵为 n_embd x n_embd
        self.c_attns = nn.ModuleList([nn.Linear(config.n_embd, config.n_embd, bias=config.bias) for _ in range(3)])
        # 输出的线性层，维度为 n_embd x n_embd
        self.c_proj = nn.Linear(config.n_embd, config.n_embd, bias=config.bias)
        # 注意力的 dropout
        self.attn_dropout = nn.Dropout(config.dropout)
        # 残差连接的 dropout
        self.resid_dropout = nn.Dropout(config.dropout)
        # 头数
        self.n_head = config.n_head
        # 隐藏层维度
        self.n_embd = config.n_embd
        # Dropout 概率
        self.dropout = config.dropout
        # 是否是解码器的 Casual LM
        self.is_causal = is_causal
        # 判断是否使用 Flash Attention，Pytorch 2.0 支持，即判断 torch.nn.functional.scaled_dot_product_attention 是否存在
        # self.flash = hasattr(torch.nn.functional, 'scaled_dot_product_attention')
        self.flash = False
        
        # 如果不使用 Flash Attention，打印一个警告
        if not self.flash:
            print("WARNING: using slow attention. Flash Attention requires PyTorch >= 2.0")
            # 如果自己实现 MHSA，需要一个 causal mask，确保 attention 只能作用在输入序列的左边
            # 此处使用 register_buffer 注册一个 bias 属性
            # bias 是一个上三角矩阵，维度为 1 x 1 x block_size x block_size，block_size 为序列最大长度
            self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size))
                                        .view(1, 1, config.block_size, config.block_size))

    def forward(self, query, key, value, attention_mask=None):
        # 输入为 query、key、value，维度为 (B, T, n_embed)
        # attention_mask 为注意力 mask，维度为 (B, 1, T)
        # print("query",query.size())
        B, _, C = key.size() # batch size, sequence length, embedding dimensionality (n_embd)
        # 计算 Q、K、V，输入通过参数矩阵层，维度为 (B, T, n_embed) x (n_embed, n_embed) -> (B, T, n_embed)
        q, k, v  = [self.c_attns[i](x) for i, x in zip(range(3), (query, key, value))]
        # 将 Q、K、V 拆分成多头，维度为 (B, T, n_head, C // n_head)，然后交换维度，变成 (B, n_head, T, C // n_head)
        k = k.view(B, -1, self.n_head, C // self.n_head).transpose(1, 2)
        q = q.view(B, -1, self.n_head, C // self.n_head).transpose(1, 2)
        v = v.view(B, -1, self.n_head, C // self.n_head).transpose(1, 2)

        # 注意力计算 
        if self.flash:
            # 直接使用 Flash Attention，其处理的是可变序列，不进行 PAD
            # 但我们在使用数据时进行了 PAD，所以会出现问题，目前暂时不考虑 PAD 带来的语义歧义
            y = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.dropout if self.training else 0, is_causal=self.is_causal)
        else:
            # 手动实现注意力计算
            # 计算 QK^T / sqrt(d_k)，维度为 (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
            att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
            if attention_mask is not None:
                # 给 attention_mask 增加一个维度
                mask = attention_mask.clone()
                mask.unsqueeze_(1)
            # 如果是解码器的 Casual LM，需要 mask 掉右上角的元素
            if self.is_causal:
                # 先对初始化的下三角矩阵做截断并转化为Bool矩阵
                casual_mask = self.bias[:,:,:k.size(-2),:k.size(-2)] == 1
                # print(casual_mask.size())
                # print(mask.size())
                # casual_mask 和原先的 attention_mask 做位运算
                if attention_mask is not None:
                    print("k: ", k.size())
                    print("q: ", q.size())
                    print("mask: ", mask.size())
                    print("casual_mask: ", casual_mask.size())
                    mask = mask & casual_mask
                else:
                    mask = casual_mask
            if attention_mask is None and not self.is_causal:
                # 不进行 Mask
                pass
            else:
                # 进行 Mask
                # print("q: ", q.size())
                # print("k: ", k.size())
                # if attention_mask is not None:
                #     print("atten_mask: ", attention_mask.size())
                # print("mask: ", mask.size())
                # print("attn: ", att.size())
                att = att.masked_fill(mask == False, float('-inf'))
            # 计算 softmax，维度为 (B, nh, T, T)
            att = F.softmax(att, dim=-1)
            # Attention Dropout
            att = self.attn_dropout(att)
            # V * Score，维度为(B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
            y = att @ v 
        # 将多头的结果拼接起来, 先交换维度为 (B, T, n_head, C // n_head)，再拼接成 (B, T, n_head * C // n_head)
        # 使用 contigonous() 函数保证内存是连续的，否则会报错
        # print(self.is_causal)
        # print(y.size())
        # print(B, T, C)
        y = y.transpose(1, 2).contiguous().view(B, -1, C)

        # 经过输出层计算，维度为 (B, T, C)，再经过线性层残差连接
        y = self.resid_dropout(self.c_proj(y))
        return y

In [4]:

'''全连接模块'''
class MLP(nn.Module):

    def __init__(self, config):
        super().__init__()
        # Transformer 的全连接模块有两个线性层，中间加了一个 RELU 激活函数
        self.c_fc    = nn.Linear(config.n_embd, 4 * config.n_embd, bias=config.bias)
        self.relu    = nn.ReLU()
        self.c_proj  = nn.Linear(4 * config.n_embd, config.n_embd, bias=config.bias)
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, x):
        x = self.c_fc(x)
        x = self.relu(x)
        x = self.c_proj(x)
        x = self.dropout(x)
        return x

In [5]:
'''层规范化模块'''

class LayerNorm(nn.Module):
    # 在 Pytorch 的 LayerNorm 基础上添加了偏置，因为 Pytorch 的 LayerNorm 不支持偏置为 None

    def __init__(self, ndim, bias):
        super().__init__()
        # 初始化参数和偏置
        self.weight = nn.Parameter(torch.ones(ndim))
        self.bias = nn.Parameter(torch.zeros(ndim)) if bias else None

    def forward(self, input):
        # 直接调用 Pytorch 的 LayerNorm
        return F.layer_norm(input, self.weight.shape, self.weight, self.bias, 1e-5)

In [6]:
'''Encoder Layer'''
class EncoderLayer(nn.Module):

    def __init__(self, config):
        super().__init__()
        # 一个 Layer 中有两个 LayerNorm，分别在 Attention 之前和 MLP 之前
        self.ln_1 = LayerNorm(config.n_embd, bias=config.bias)
        # Encoder 不需要掩码，传入 is_causal=False
        self.attn = MultiHeadAttention(config, is_causal=False)
        self.ln_2 = LayerNorm(config.n_embd, bias=config.bias)
        self.mlp = MLP(config)

    def forward(self, x, attn_mask=None):
        # 此处前面加了 x 实则是实现了残差连接
        x = self.ln_1(x)
        # Encoder 使用 Self Attention，所以 Q、K、V 都是 x
        # print("x",x.size())
        x = x + self.attn(x, x, x, attention_mask=attn_mask)
        x = x + self.mlp(self.ln_2(x))
        return x

In [7]:
'''Encoder'''
class Encoder(nn.Module):

    def __init__(self, config):
        super(Encoder, self).__init__() 
        # 一个 Encoder 由 N 个 Encoder Layer 组成
        self.layers = nn.ModuleList([EncoderLayer(config) for _ in range(config.n_layer)])
        self.norm = LayerNorm(config.n_embd, bias=config.bias)

    def forward(self, x, attn_mask=None):
        "分别通过 N 层 Encoder Layer"
        for layer in self.layers:
            x = layer(x, attn_mask=attn_mask)
        return self.norm(x)

In [8]:
'''Decoder Layer'''
class DecoderLayer(nn.Module):

    def __init__(self, config):
        super().__init__()
        # 一个 Layer 中有三个 LayerNorm，分别在 Mask Attention 之前、Self Attention 之前和 MLP 之前
        self.ln_1 = LayerNorm(config.n_embd, bias=config.bias)
        # Decoder 的第一个部分是 Mask Attention，传入 is_causal=True
        self.m_attn = MultiHeadAttention(config, is_causal=True)
        self.ln_2 = LayerNorm(config.n_embd, bias=config.bias)
        # Decoder 的第二个部分是 类似于 Encoder 的 Attention，传入 is_causal=False
        self.attn = MultiHeadAttention(config, is_causal=False)
        self.ln_3 = LayerNorm(config.n_embd, bias=config.bias)
        # 第三个部分是 MLP
        self.mlp = MLP(config)

    def forward(self, x, enc_out, attn_mask=None, label_mask=None):
        # 此处前面加了 x 实则是实现了残差连接
        x = self.ln_1(x)
        # 第一部分是一个 Mask Self Attention，Q、K、V 都是 x
        x = x + self.m_attn(x, x, x, attention_mask=label_mask)
        x = self.ln_2(x)
        # 第二部分是一个类似于 Encoder 的 Attention，Q 是 x，K、V 是 Encoder 的输出
        x = x + self.attn(x, enc_out, enc_out, attention_mask=attn_mask)
        x = self.ln_3(x)
        x = x + self.mlp(x)
        return x

In [9]:
'''Decoder'''
class Decoder(nn.Module):

    def __init__(self, config):
        super(Decoder, self).__init__() 
        # 一个 Decoder 由 N 个 Decoder Layer 组成
        self.layers = nn.ModuleList([DecoderLayer(config) for _ in range(config.n_layer)])
        self.norm = LayerNorm(config.n_embd, bias=config.bias)

    def forward(self, x, enc_out, attn_mask=None, label_mask=None):
        "Pass the input (and mask) through each layer in turn."
        for layer in self.layers:
            x = layer(x, enc_out, attn_mask=attn_mask, label_mask=label_mask)
        return self.norm(x)

In [10]:
'''位置编码模块'''
class PositionalEncoding(nn.Module):
    # 在输入上加入了位置编码

    def __init__(self, config):
        super(PositionalEncoding, self).__init__()
        # Dropout 层
        self.dropout = nn.Dropout(p=config.dropout)

        # block size 是序列的最大长度
        pe = torch.zeros(config.block_size, config.n_embd)
        position = torch.arange(0, config.block_size).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, config.n_embd, 2) * -(math.log(10000.0) / config.n_embd)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x):
        x = x + self.pe[:, : x.size(1)].requires_grad_(False)
        return self.dropout(x)

In [11]:
import inspect

'''整体模型'''
class Transformer(nn.Module):

    def __init__(self, config):
        super().__init__()
        # 必须输入词表大小和 block size
        assert config.vocab_size is not None
        assert config.block_size is not None
        self.config = config
        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            wpe = PositionalEncoding(config),
            drop = nn.Dropout(config.dropout),
            encoder = Encoder(config),
            decoder = Decoder(config),
        ))
        # 最后的线性层，输入是 n_embd，输出是词表大小
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        # 初始化所有的权重
        self.apply(self._init_weights)

        # 查看所有参数的数量
        print("number of parameters: %.2fM" % (self.get_num_params()/1e6,))

    '''统计所有参数的数量'''
    def get_num_params(self, non_embedding=False):
        # non_embedding: 是否统计 embedding 的参数
        n_params = sum(p.numel() for p in self.parameters())
        # 如果不统计 embedding 的参数，就减去
        if non_embedding:
            n_params -= self.transformer.wpe.weight.numel()
        return n_params

    '''初始化权重'''
    def _init_weights(self, module):
        # 线性层和 Embedding 层初始化为正则分布
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    
    '''前向计算函数'''
    def forward(self, idx, targets, attn_mask=None, label_mask=None):
        # 输入为 idx，维度为 (batch size, sequence length)；targets 为目标序列，用于计算 loss
        device = idx.device
        b, t = idx.size()
        assert t <= self.config.block_size, f"不能计算该序列，该序列长度为 {t}, 最大序列长度只有 {self.config.block_size}"

        # 通过 self.transformer
        # 首先将输入 idx 通过 Embedding 层，得到维度为 (batch size, sequence length, n_embd)
        print("idx",idx.size())
        # 通过 Embedding 层得到的维度是 (batch size, sequence length, vocab_size, n_embd)，因此我们去掉倒数第二个维度
        tok_emb = self.transformer.wte(idx)
        label_emb = self.transformer.wte(targets)
        print("tok_emb",tok_emb.size())
        # 然后通过位置编码
        pos_emb = self.transformer.wpe(tok_emb) 
        labels_pos_emb = self.transformer.wpe(label_emb)
        # 再进行 Dropout
        x = self.transformer.drop(pos_emb)
        # 然后通过 Encoder
        print("x after wpe:",x.size())
        enc_out = self.transformer.encoder(x, attn_mask=attn_mask)
        print("enc_out:",enc_out.size())
        # 再通过 Decoder
        x = self.transformer.decoder(labels_pos_emb, enc_out, attn_mask=attn_mask, label_mask=label_mask)
        print("x after decoder:",x.size())

        if targets is not None:
            # 训练阶段，如果我们给了 targets，就计算 loss
            # 先通过最后的 Linear 层，得到维度为 (batch size, sequence length, vocab size)
            logits = self.lm_head(x)
            print("logits: ", logits.size())
            print("targets: ", targets.size())
            # 再跟 targets 计算交叉熵
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)
        else:
            # 推理阶段，我们只需要 logits，loss 为 None
            # 取 -1 是只取序列中的最后一个作为输出
            logits = self.lm_head(x[:, [-1], :]) # note: using list [-1] to preserve the time dim
            loss = None

        return logits, loss

    '''配置优化器'''
    def configure_optimizers(self, weight_decay, learning_rate, betas, device_type):
        # weight_decay: 权重衰减系数，learning_rate: 学习率，betas: AdamW 的 betas，device_type: 设备类型
        # 首先获取所有命名参数
        param_dict = {pn: p for pn, p in self.named_parameters()}
        # 过滤掉不需要更新的参数
        param_dict = {pn: p for pn, p in param_dict.items() if p.requires_grad}
        # 参数根据维度分为两组。
        # 维度大于等于2的参数（通常是权重）会应用权重衰减，而维度小于2的参数（通常是偏置和层归一化参数）不会应用权重衰减。
        decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
        nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
        optim_groups = [
            {'params': decay_params, 'weight_decay': weight_decay},
            {'params': nodecay_params, 'weight_decay': 0.0}
        ]
        # 打印一下参数数量
        num_decay_params = sum(p.numel() for p in decay_params)
        num_nodecay_params = sum(p.numel() for p in nodecay_params)
        print(f"应用权重衰减的层数: {len(decay_params)}； 总参数量为：{num_decay_params:,}")
        print(f"不应用权重衰减的层数: {len(nodecay_params)}, 总参数量为：{num_nodecay_params:,}")
        # 检查 torch.optim.AdamW 是否支持融合版本（fused version），这是针对 CUDA 设备优化的版本。如果可用且 device_type 为 'cuda'，则使用融合版本。
        fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
        use_fused = fused_available and device_type == 'cuda'
        extra_args = dict(fused=True) if use_fused else dict()
        # 创建优化器
        optimizer = torch.optim.AdamW(optim_groups, lr=learning_rate, betas=betas, **extra_args)
        print(f"是否使用 fused AdamW: {use_fused}")

        return optimizer

    '''进行推理'''
    @torch.no_grad()
    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
        # 推理阶段，输入为 idx，维度为 (batch size, sequence length)，max_new_tokens 为最大生成的 token 数量即按序推理 max_new_tokens 次
        for _ in range(max_new_tokens):
            # 如果输入序列太长，我们需要将它截断到 block_size
            idx_cond = idx if idx.size(1) <= self.config.block_size else idx[:, -self.config.block_size:]
            # 前向计算，得到 logits，维度为 (batch size, sequence length, vocab size)
            logits, _ = self(idx_cond, idx_cond)
            # 使用最后一个 token 的 logits 作为当前输出，除以温度系数控制其多样性
            logits = logits[:, -1, :] / temperature
            # 如果使用 Top K 采样，将 logits 中除了 top_k 个元素的概率置为 0
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float('Inf')
            # 对输出结果进行 Softmax
            probs = F.softmax(logits, dim=-1)
            # 对结果概率进行采样
            idx_next = torch.multinomial(probs, num_samples=1)
            # 将输出结果拼接到输入序列后面，作为下一次的输入
            idx = torch.cat((idx, idx_next), dim=1)
            # print("idx:", idx)

        return idx

In [12]:
from dataclasses import dataclass

@dataclass
class TransformerConfig:
    block_size: int = 1024
    vocab_size: int = 50304 
    n_layer: int = 4
    n_head: int = 4
    n_embd: int = 768
    dropout: float = 0.0
    bias: bool = True #

In [13]:
model_config = TransformerConfig(vocab_size=32000, block_size=128, n_layer=2, n_head=4, n_embd=16, dropout=0.0, bias=True)
model = Transformer(model_config)

number of parameters: 1.04M


In [14]:
idx = torch.randint(1, 10, (4, 8))
label = torch.randint(1, 10, (4, 8))
attn_mask = torch.ones((4, 1, 8)) == 1
logits, _ = model(idx, label, attn_mask=attn_mask)
print("logits",logits.size())

idx torch.Size([4, 8])
tok_emb torch.Size([4, 8, 16])
x after wpe: torch.Size([4, 8, 16])
enc_out: torch.Size([4, 8, 16])
x after decoder: torch.Size([4, 8, 16])
logits:  torch.Size([4, 8, 32000])
targets:  torch.Size([4, 8])
logits torch.Size([4, 8, 32000])


In [15]:
result = model.generate(idx, 3)
print("generate result",result.size())

idx torch.Size([4, 8])
tok_emb torch.Size([4, 8, 16])
x after wpe: torch.Size([4, 8, 16])
enc_out: torch.Size([4, 8, 16])
x after decoder: torch.Size([4, 8, 16])
logits:  torch.Size([4, 8, 32000])
targets:  torch.Size([4, 8])
idx torch.Size([4, 9])
tok_emb torch.Size([4, 9, 16])
x after wpe: torch.Size([4, 9, 16])
enc_out: torch.Size([4, 9, 16])
x after decoder: torch.Size([4, 9, 16])
logits:  torch.Size([4, 9, 32000])
targets:  torch.Size([4, 9])
idx torch.Size([4, 10])
tok_emb torch.Size([4, 10, 16])
x after wpe: torch.Size([4, 10, 16])
enc_out: torch.Size([4, 10, 16])
x after decoder: torch.Size([4, 10, 16])
logits:  torch.Size([4, 10, 32000])
targets:  torch.Size([4, 10])
generate result torch.Size([4, 11])


In [16]:
result

tensor([[    1,     9,     5,     4,     6,     7,     2,     7, 25412, 20609,
           461],
        [    6,     9,     2,     1,     2,     6,     6,     7, 16516,  6875,
         20417],
        [    9,     4,     1,     1,     6,     5,     3,     7,  1033, 31221,
         21766],
        [    1,     2,     1,     9,     6,     1,     5,     6,  1287, 10562,
          4435]])

## 训练数据构造

In [17]:
import json

data = json.load(open("/root/data/model/wmt/dev.json", "r", encoding="utf-8"))
data

[['Some analysts argue that the negative effects of such an outcome would only last for “months.”',
  '某些分析家认为军事行动的负面效果只会持续短短“几个月”。'],
 ['The Fed apparently could not stomach the sell-off in global financial markets in January and February, which was driven largely by concerns about further tightening.',
  '美联储显然无法消化1月和2月的全球金融市场抛售，而这一抛售潮主要是因为对美联储进一步紧缩的担忧导致的。'],
 ['Renewing the South Korean Miracle', '再造韩国奇迹'],
 ['He had no doubt about which vision would win out: “Yearly one nation after another would drop into the union which best suited it; and looking to the commercial activity of the Teutonic races, and the comparative torpor of the Latin races, no doubt the Teutonic money would be most frequently preferred.”',
  '他对哪种货币会胜出坚信不疑：“假以时日，一个有一个国家会进入最适合它的联盟； 看看条顿竞赛中的商业活动，再看看拉丁竞赛中的死气沉沉，毫无疑问条顿货币将是最受欢迎的。'],
 ['Today, however, the US military is more likely to be used to stabilize and rebuild failed states, assist partners in countering insurgency and terrorism, control nuclear weapons when r

In [18]:
import os

file_path = "/root/data/model/wmt"
files = ['train', 'dev', 'test']
ch_path = 'corpus.ch'
en_path = 'corpus.en'
ch_lines = []
en_lines = []

for file in files:
    corpus = json.load(open(os.path.join(file_path, file + '.json'), 'r'))
    for item in corpus:
        ch_lines.append(item[1] + '\n')
        en_lines.append(item[0] + '\n')

with open(os.path.join(file_path, ch_path), "w") as fch:
    fch.writelines(ch_lines)

with open(os.path.join(file_path, en_path), "w") as fen:
    fen.writelines(en_lines)

# lines of Chinese: 252777
print("lines of Chinese: ", len(ch_lines))
# lines of English: 252777
print("lines of English: ", len(en_lines))
print("-------- Get Corpus ! --------")

lines of Chinese:  252777
lines of English:  252777
-------- Get Corpus ! --------


## 训练分词器

import sentencepiece as spm


def train(input_file, vocab_size, model_name, model_type, character_coverage):
    # 使用 Sentence Piece 基于训练数据来训练一个分词器
    # args:
    # input_file: 训练使用的数据
    # vocab_size: 设定的词表大小
    # model_name: 模型命名
    # model_type: 模型类型，一般选择 bpe
    # character_coverage: 覆盖的字符范围，中文一类的表意文字一般0.995，英文一类的字母文字一般1
    # 采用命令行的形式实现
    input_argument = '--input=%s --model_prefix=%s --vocab_size=%s --model_type=%s --character_coverage=%s ' \
                     '--pad_id=0 --unk_id=1 --bos_id=2 --eos_id=3 '
    cmd = input_argument % (input_file, model_name, vocab_size, model_type, character_coverage)
    spm.SentencePieceTrainer.Train(cmd)


en_input = '/root/data/model/wmt/corpus.en'
en_vocab_size = 32000
en_model_name = 'eng'
en_model_type = 'bpe'
en_character_coverage = 1
train(en_input, en_vocab_size, en_model_name, en_model_type, en_character_coverage)

ch_input = '/root/data/model/wmt/corpus.ch'
ch_vocab_size = 32000
ch_model_name = 'chn'
ch_model_type = 'bpe'
ch_character_coverage = 0.9995
train(ch_input, ch_vocab_size, ch_model_name, ch_model_type, ch_character_coverage)


In [19]:
# 加载训练好的分词器
import sentencepiece as spm

def chinese_tokenizer_load():
    sp_chn = spm.SentencePieceProcessor()
    sp_chn.Load('{}.model'.format("/root/data/logan/through_pytorch/chn"))
    return sp_chn


def english_tokenizer_load():
    sp_eng = spm.SentencePieceProcessor()
    sp_eng.Load('{}.model'.format("/root/data/logan/through_pytorch/eng"))
    return sp_eng

## 定义 Dataset


In [20]:
from torch.utils.data import Dataset
import json
from torch.nn.utils.rnn import pad_sequence
import numpy as np

class MTDataset(Dataset):
    def __init__(self, data_path):
        self.out_en_sent, self.out_ch_sent = self.get_dataset(data_path, sort=True)
        self.sp_eng = english_tokenizer_load()
        self.sp_chn = chinese_tokenizer_load()
        self.PAD = self.sp_eng.pad_id()  # 0
        self.BOS = self.sp_eng.bos_id()  # 2
        self.EOS = self.sp_eng.eos_id()  # 3

    @staticmethod
    def len_argsort(seq):
        """传入一系列句子数据(分好词的列表形式)，按照句子长度排序后，返回排序后原来各句子在数据中的索引下标"""
        return sorted(range(len(seq)), key=lambda x: len(seq[x]))

    def get_dataset(self, data_path, sort=False):
        """把中文和英文按照同样的顺序排序, 以英文句子长度排序的(句子下标)顺序为基准"""
        # 加载数据集
        dataset = json.load(open(data_path, 'r'))
        # 将中文和英文分别加载为两个列表
        out_en_sent = []
        out_ch_sent = []
        for idx, _ in enumerate(dataset):
            out_en_sent.append(dataset[idx][0])
            out_ch_sent.append(dataset[idx][1])
        # 如果要按长度排序
        if sort:
            sorted_index = self.len_argsort(out_en_sent)
            out_en_sent = [out_en_sent[i] for i in sorted_index]
            out_ch_sent = [out_ch_sent[i] for i in sorted_index]
        return out_en_sent, out_ch_sent

    def __getitem__(self, idx):
        # get 方法，返回一个句对
        eng_text = self.out_en_sent[idx]
        chn_text = self.out_ch_sent[idx]
        return [eng_text, chn_text]

    def __len__(self):
        return len(self.out_en_sent)

    def collate_fn(self, batch):
        # 变长序列的 collate_fn 方法，需要进行 padding
        # 形成列表
        src_text = [x[0] for x in batch]
        tgt_text = [x[1] for x in batch]
        # 进行 tokenizer，然后加上 BOS 和 EOS
        src_tokens = [[self.BOS] + self.sp_eng.EncodeAsIds(sent) + [self.EOS] for sent in src_text]
        tgt_tokens = [[self.BOS] + self.sp_chn.EncodeAsIds(sent) + [self.EOS] for sent in tgt_text]
        # 进行 padding
        batch_input = pad_sequence([torch.LongTensor(np.array(l_)) for l_ in src_tokens],
                                   batch_first=True, padding_value=self.PAD)
        batch_target = pad_sequence([torch.LongTensor(np.array(l_)) for l_ in tgt_tokens],
                                    batch_first=True, padding_value=self.PAD)
        src_mask = (batch_input != self.PAD).unsqueeze(-2)
        label_mask = (batch_target != self.PAD).unsqueeze(-2)
        return {"input_ids": batch_input, "label_ids": batch_target, "attention_mask": src_mask, "label_mask": label_mask}
        # return {"input_ids": batch_input, "attention_mask": src_mask}

In [21]:
dataset = MTDataset("/root/data/model/wmt/dev.json")

In [22]:
dataloader = torch.utils.data.DataLoader(dataset, shuffle=True, batch_size=4, collate_fn=dataset.collate_fn)

In [23]:
for item in dataloader:
    print(item)
    break

{'input_ids': tensor([[    2,  2668,  2338,   236,  3806,    39,  2186,  6506, 31841,   323,
           406,   490,    55,     6,  6157,  4853,    81,    10,   371,  4692,
            62,  1106,    10,  1639, 31843,     3,     0,     0,     0,     0,
             0,     0],
        [    2, 24876, 31847,  4431,  7941,    59,     6,  6716,  3756,    31,
            10,  2314,  1830,    62,   441,    10,   277,   146,   365,   345,
           436,  8355, 31841,    80,   236,  1759,   365,   345,   436,  4428,
         31843,     3],
        [    2,    99,  1277,    62,  6951,  1235,    59, 26421,  6364,    59,
            62,    80, 24979,  5146,    62,   222,  2198,   721,    10,  6951,
            11,    39,    10,  1766, 30074, 31843,     3,     0,     0,     0,
             0,     0],
        [    2,   402,   435,  2174,    31,   673,  2190,    26,  1067,    31,
          1580,    10,   757,  2557, 31843,     3,     0,     0,     0,     0,
             0,     0,     0,     0,     0,  

In [24]:
print("inputs_id", item["input_ids"].size())
print("label_ids", item["label_ids"].size())
print("attention_mask", item["attention_mask"].size())

inputs_id torch.Size([4, 32])
label_ids torch.Size([4, 26])
attention_mask torch.Size([4, 1, 32])


In [26]:
inputs = item["input_ids"]
attn_mask = item["attention_mask"]
labels = item["label_ids"]
label_mask = item["label_mask"]
logits, _ = model(inputs, labels, attn_mask=attn_mask, label_mask=label_mask)

idx torch.Size([4, 32])
tok_emb torch.Size([4, 32, 16])
x after wpe: torch.Size([4, 32, 16])
enc_out: torch.Size([4, 32, 16])
k:  torch.Size([4, 4, 26, 4])
q:  torch.Size([4, 4, 26, 4])
mask:  torch.Size([4, 1, 1, 26])
casual_mask:  torch.Size([1, 1, 26, 26])
k:  torch.Size([4, 4, 26, 4])
q:  torch.Size([4, 4, 26, 4])
mask:  torch.Size([4, 1, 1, 26])
casual_mask:  torch.Size([1, 1, 26, 26])
x after decoder: torch.Size([4, 26, 16])
logits:  torch.Size([4, 26, 32000])
targets:  torch.Size([4, 26])


In [29]:
model.generate(inputs, 10)

idx torch.Size([4, 32])
tok_emb torch.Size([4, 32, 16])
x after wpe: torch.Size([4, 32, 16])
enc_out: torch.Size([4, 32, 16])
x after decoder: torch.Size([4, 32, 16])
logits:  torch.Size([4, 32, 32000])
targets:  torch.Size([4, 32])
idx torch.Size([4, 33])
tok_emb torch.Size([4, 33, 16])
x after wpe: torch.Size([4, 33, 16])
enc_out: torch.Size([4, 33, 16])
x after decoder: torch.Size([4, 33, 16])
logits:  torch.Size([4, 33, 32000])
targets:  torch.Size([4, 33])
idx torch.Size([4, 34])
tok_emb torch.Size([4, 34, 16])
x after wpe: torch.Size([4, 34, 16])
enc_out: torch.Size([4, 34, 16])
x after decoder: torch.Size([4, 34, 16])
logits:  torch.Size([4, 34, 32000])
targets:  torch.Size([4, 34])
idx torch.Size([4, 35])
tok_emb torch.Size([4, 35, 16])
x after wpe: torch.Size([4, 35, 16])
enc_out: torch.Size([4, 35, 16])
x after decoder: torch.Size([4, 35, 16])
logits:  torch.Size([4, 35, 32000])
targets:  torch.Size([4, 35])
idx torch.Size([4, 36])
tok_emb torch.Size([4, 36, 16])
x after wpe:

tensor([[    2,  2668,  2338,   236,  3806,    39,  2186,  6506, 31841,   323,
           406,   490,    55,     6,  6157,  4853,    81,    10,   371,  4692,
            62,  1106,    10,  1639, 31843,     3,     0,     0,     0,     0,
             0,     0, 14135,  4281, 10522, 21702, 12728, 30598,  3686, 12030,
         15986,  3309],
        [    2, 24876, 31847,  4431,  7941,    59,     6,  6716,  3756,    31,
            10,  2314,  1830,    62,   441,    10,   277,   146,   365,   345,
           436,  8355, 31841,    80,   236,  1759,   365,   345,   436,  4428,
         31843,     3, 28800,  4633, 22366, 20202, 28156, 19610, 20725,  2965,
         17115, 16764],
        [    2,    99,  1277,    62,  6951,  1235,    59, 26421,  6364,    59,
            62,    80, 24979,  5146,    62,   222,  2198,   721,    10,  6951,
            11,    39,    10,  1766, 30074, 31843,     3,     0,     0,     0,
             0,     0,  3927, 26351, 16965, 20576, 29083, 28264, 27495,  7917,
    